# Libraries

In [229]:
import math
import random
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Module 1 + Module 2
By combining *Module 1* and *Module 2* the problem is formally defined as follows:
- a depot $0$ where all vehicles are located in the beginning
- a set of customers $C = {1,...,N}$, we define locations $L = {0} + C$
- a demand $d_i$ for each customer $i \in C$ (the depot has no demand)
- a time window $[a_i, b_i]$ with $0<a_i<b_i$ for each location $i \in L$ (including the depot)
- a service time $s_i$ for each customer $i \in C$ (the depot has zero service time)
- costs $c_{ij}$ and time $t_{ij}$ for travelling from location $i$ to location $j$
- a homogeneous fleet $K = {1,...,|K|}$ of vehicles with load capacity $Q$

The **objective** is:
- to find a route for each vehicle, i.e., an ordered sequence of customers
- with minimal total routing time, such that:
    - each customer is visited exactly once
    - the start times of the services have to be within the depot and customer time windows (potentially introducing waiting times between services),
    - and the vehicle capacity is never exceeded

## Input Configuration
We configurate the input values:

In [230]:
numCustomers = 5
maxNumVehicles = 2

vehicleCapacity = 30
demandRange = (2, 5)

timeHorizon = 100 # tmax
timeWindowWidthRange = (10, 15)
serviceTimeRange = (2, 4)

## Instance Generation
By using the values in the *Input Configuration*, we generate the instances of the problem:

In [231]:
random.seed(0)

depot = 0
customers = [*range(1, numCustomers + 1)] # array of customers
locations = [depot] + customers # depot + customers
connections = [(i, j) for i in locations for j in locations if i != j] # set of all possible edges
vehicles = [*range(1, maxNumVehicles + 1)] # array of vehicles

# create random depot and customer locations in the Euclidian plane (1000x1000)
points = [(random.randint(0, 999), random.randint(0, 999)) for i in locations]
# dictionary of Euclidean distance for each connection (interpreted as travel costs)
costs = {
    (i, j): math.ceil(
        math.sqrt(sum((points[i][k] - points[j][k]) ** 2 for k in range(2)))
    )
    for (i, j) in connections
}
maximalCosts = math.ceil(999 * math.sqrt(2))

# dictionary of travel times for each connection (related to the costs, scaled to time horizon)
travelTimes = {
    (i, j): math.ceil((costs[i, j] / maximalCosts) * timeHorizon * 0.2)
    for (i, j) in connections
}

# create random demands
demands = {i: random.randint(demandRange[0], demandRange[1]) for i in customers}
demands[0] = 0  # depot has no demand

# create random service times,
serviceTimes = {
    i: random.randint(serviceTimeRange[0], serviceTimeRange[1]) for i in customers
}
serviceTimes[0] = 0  # depot has no service time

# create random time windows
timeWindowWidths = {
    i: random.randint(timeWindowWidthRange[0], timeWindowWidthRange[1])
    for i in customers
}

# vehicles are allowed to leave the depot any time within the time horizon
timeWindowWidths[0] = timeHorizon

# create time windows randomly based on the previously generated information
# such that the service at a customer can be finished within the time horizon
timeWindows = {}
timeWindows[0] = (0, 0 + timeWindowWidths[0])
for i in customers:
    start = random.randint(0, timeHorizon - serviceTimes[i] - timeWindowWidths[i] - travelTimes[i,0])
    timeWindows[i] = (start, start + timeWindowWidths[i])

So now, we can print all these parameters:

In [232]:
print("Customer Demands:")
for customer, demand in demands.items():
    print(f"Customer {customer}: Demand = {demand}")

print("\nCustomer Service Times:")
for customer, service_time in serviceTimes.items():
    print(f"Customer {customer}: Service Time = {service_time}")

print("\nCustomer Time Windows:")
for customer, window in timeWindows.items():
    print(f"Customer {customer}: Time Window = {window}")

print("\nTravel Times:")
for (i, j), time in list(travelTimes.items()):
    print(f"Travel Time from {i} to {j}: {time}")

Customer Demands:
Customer 1: Demand = 4
Customer 2: Demand = 5
Customer 3: Demand = 4
Customer 4: Demand = 3
Customer 5: Demand = 3
Customer 0: Demand = 0

Customer Service Times:
Customer 1: Service Time = 3
Customer 2: Service Time = 2
Customer 3: Service Time = 2
Customer 4: Service Time = 4
Customer 5: Service Time = 3
Customer 0: Service Time = 0

Customer Time Windows:
Customer 0: Time Window = (0, 100)
Customer 1: Time Window = (12, 26)
Customer 2: Time Window = (9, 24)
Customer 3: Time Window = (42, 56)
Customer 4: Time Window = (60, 71)
Customer 5: Time Window = (71, 83)

Travel Times:
Travel Time from 0 to 1: 8
Travel Time from 0 to 2: 8
Travel Time from 0 to 3: 12
Travel Time from 0 to 4: 6
Travel Time from 0 to 5: 11
Travel Time from 1 to 0: 8
Travel Time from 1 to 2: 14
Travel Time from 1 to 3: 8
Travel Time from 1 to 4: 7
Travel Time from 1 to 5: 6
Travel Time from 2 to 0: 8
Travel Time from 2 to 1: 14
Travel Time from 2 to 3: 14
Travel Time from 2 to 4: 7
Travel Time fr

## Model Generation
In this section we will see two different mathematical formulations of this problem:
- Big - M Model
- Flow Model

In particular, the two models just mentioned share the same basic model (Vehicle Routing Problem), but differ with regard to capacity and time constraints.

### Big - M Model
**Decision Variables**:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$
- $y_i$ for each location $i \in L$ to denote the **vehicle load after picking up the demand at $i$**.
- $z_i$ for each location $i \in L$ to denote the **start time of the service at $i$** that has to be within its time window

In [233]:
model_M = gp.Model("VRPTW")

x = model_M.addVars(connections, vtype=GRB.BINARY, name="x")

y = model_M.addVars(locations, lb=0, ub=vehicleCapacity, name="y")
y[0].UB = 0

z = model_M.addVars(locations, name="z")
for i in locations:
    z[i].LB = timeWindows[i][0] 
    z[i].UB = timeWindows[i][1]

**Objective Function:**

Minimize the total travel time. Travel time doesn't consider the service time of each customer:
$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

In [234]:
model_M.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

**Constraints:**
- Each customer is visited exactly once, i.e., there is exactly one incoming connection and exactly one    outgoing connection:
$$
\sum_{j \in \mathcal{L}} x_{ij} = 1 \quad \forall i \in \mathcal{C}
$$

$$
\sum_{i \in \mathcal{L}} x_{ij} = 1 \quad \forall j \in \mathcal{C}
$$

In [235]:
model_M.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model_M.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

- At most $|K|$ vehicles can be used:
$$
\sum_{j \in \mathcal{C}} x_{0j} \leq |K|
$$

In [236]:
model_M.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")

<gurobi.Constr *Awaiting Model Update*>

- We can also compute a lower bound on the number of needed vehicles, that is the sum of all demands divided by the vehicle capacity:
$$
\sum_{j \in \mathcal{C}} x_{0j} \geq \left\lceil \frac{\sum_{i \in \mathcal{C}} d_i}{Q} \right\rceil
$$

In [237]:
def numVehiclesNeededForCustomers(customers):
    sumDemand = 0
    for i in customers:
        sumDemand += demands[i]
    return math.ceil(sumDemand / vehicleCapacity)

model_M.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

<gurobi.Constr *Awaiting Model Update*>

- The current capacity of the vehicle must not exceed the maximum capacity of the vehicle:
    $$y_i + d_j \le y_j + Q(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$$
    In particular:
    - if $x_{ij} = 1$: the vehicle travels from $i$ to $j$. So, $y_j$ must be equal or greater than $y_i + d_j$
    - if $x_{ij} = 0$: it ensures that vehicle capacity si not exceed.

In [238]:
model_M.addConstrs(
    (
        y[i] + demands[j] <= y[j] + vehicleCapacity * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM1",
)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


- This constraint ensures that, if arc $(i, j)$ is travelled then the load at node $j$ must be greater or equal to $y_i + d_j$:
    $$y_i + d_j \geq y_j - M_{ij}(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$$
    In particular, if the arc $(i, j)$ is not travelled then the inequality becomes non-binding since $M$ is a big constant

In [239]:
model_M.addConstrs(
    (
        y[i] + demands[j]
        >= y[j] - (vehicleCapacity - demands[i] - demands[j]) * (1 - x[i, j])
        for i in locations
        for j in customers
        if i != j
    ),
    name="loadBigM2",
)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


- This constraint manages the time required to pass from a node $i$ to a node $j$, taking into account the service time at node $i$ ($s(i)$) and the travel time between $i$ and $j$ ($t_{ij}$):
    $$z_i + s_i + t_{ij} \leq z_j + T(1 - x_{ij}) \quad \forall i \in L, j \in C, i \neq j$$
    In particular, if the arc $(i, j)$ is not travelled, then $x_{ij} = 0$ and the constraint is not activated because of the big term $T$.

In [240]:
model_M.addConstrs(
        (
            z[i] + serviceTimes[i] + travelTimes[i, j]
            <= z[j]
            + (
                timeWindows[i][1]
                + serviceTimes[i]
                + travelTimes[i, j]
                - timeWindows[j][0]
            )
            * (1 - x[i, j])
            for i in locations
            for j in customers
            if i != j
        ),
        name="timeBigM",
    )

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,


So, we can now optimize the model and plot the solution:

In [241]:
model_M.Params.OutputFlag = 0

model_M.optimize()

if model_M.SolCount >= 1:

    print("Objective Value: ", model_M.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        #time = travelTimes[0, firstCustomer]
        #violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            #time = max(time, timeWindows[currentCustomer][0])
            #if time > timeWindows[currentCustomer][1]:
                #violatedTimeWindows = True
            #time += (
                #serviceTimes[currentCustomer]
                #+ travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            #)
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        #if violatedTimeWindows:
            #print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")
    print()
    for c in customers: 
        print(f"Client {c}: service start time = {model_M.getAttr('x', z)[c]}, time window = {timeWindows[c]}")

Objective Value:  51.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0) -> 3 (L:4) -> 5 (L:8) -> 0 (L:11/30)
Route 1: 0 -> 2 (L:0) -> 4 (L:5) -> 0 (L:8/30)
Unvisited customers: 
Client 1: service start time = 26.0, time window = (12, 26)
Client 2: service start time = 9.0, time window = (9, 24)
Client 3: service start time = 42.0, time window = (42, 56)
Client 4: service start time = 60.0, time window = (60, 71)
Client 5: service start time = 83.0, time window = (71, 83)


### Flow Model
Now, we extend the meaning of the variables used in the Big-M model. We do not associate resource states to customers anymore but instead to connections between customers.

**Decision Variables:**
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$
- $y_{ij}$ for each connection $(i, j)$ to denote the vehicle load after picking up the demand at $i$ and proceeding to location $j$.
- $z_{ij}$ for each connection $(i, j)$ to denote the start time of the service at $i$ (which must be within its time window) when immediately proceeding to location $j$.



In [242]:
model_flow = gp.Model("VRPTW")

x = model_flow.addVars(connections, vtype=GRB.BINARY, name="x")

y = model_flow.addVars(connections, lb=0, ub=vehicleCapacity, name="y")

for i in customers:
    y[0, i].UB = 0

z = model_flow.addVars(connections, lb=0, name="z")

for (i, j) in connections:
    z[i, j].UB = timeWindows[i][1]

**Objective Function:**

Minimize the total travel time. Travel time doesn't consider the service time of each customer:
$$
\text{min} \sum_{i,j \in \mathcal{L}} c_{ij}x_{ij}
$$

In [243]:
model_flow.setObjective(gp.quicksum(travelTimes[i,j]*x[i,j] for i,j in connections), GRB.MINIMIZE)

**Constraints:**

As stated above, the two models only differ in terms of capacity and time constraints. Consequently, the constraints imposed on the basic model (VRP), i.e. on the decision variable x remain unchanged.

In [244]:
model_flow.addConstrs((x.sum("*", j) == 1 for j in customers), name="incoming")
model_flow.addConstrs((x.sum(i, "*") == 1 for i in customers), name="outgoing")

model_flow.addConstr(x.sum(0, "*") <= maxNumVehicles, name="maxNumVehicles")

model_flow.addConstr(
    x.sum(0, "*") >= numVehiclesNeededForCustomers(customers),
    name="minNumVehicles",
)

<gurobi.Constr *Awaiting Model Update*>

- This constraint ensures that the load of the vehicle arriving at node $j$ plus the satisfied demand $d_j$ is equal to the load leaving node $j$. In other words, when the vehicle satisfies customer demand $j$, the load decreases by the amount $d_j$ and continues with the remaining load.
    $$\sum_{i \in L} y_{ij} + d_j = \sum_{i \in L} y_{ji} \quad \forall j \in C$$

In [245]:
model_flow.addConstrs(
    (y.sum("*", j) + demands[j] == y.sum(j, "*") for j in customers),
    name="flowConservation",
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

- This constraint ensures that the vehicle, if it travels from location $i$ to $j$ ($x_{ij} = 1$), carries at least the load demanded by $i$. Thus, if the vehicle visits $i$ and goes to $j$, it must carry at least $d_i$, which is the demand of node $i$.
    $$y_{ij} \geq d_ix_{ij} \quad \forall i \in C, j \in L, i \neq j$$

In [246]:
model_flow.addConstrs(
    (
        y[i, j] >= demands[i] * x[i, j]
        for i in customers
        for j in locations
        if i != j
    ),
    name="loadLowerBound",
)

{(1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


- This constraint limits the maximum load a vehicle can carry over an arc $(i,j)$. The vehicle cannot carry more than $Q - d_j$, where $Q$ is the maximum capacity of the vehicle and $d_j$ is the demand of customer $j$. If the vehicle travels the arc $(i,j)$, then $y_{ij}$ must be less than or equal to the maximum allowed load.
    $$y_{ij} \leq (Q - d_j)x_{ij} \quad \forall i \in C, j \in L, i \neq j$$

In [247]:
model_flow.addConstrs(
    (
        y[i, j] <= (vehicleCapacity - demands[j]) * x[i, j]
        for i in customers
        for j in locations
        if i != j
    ),
    name="loadUpperBound",
)

{(1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


- If the vehicle travels from location $i$ to $j$, the service start time at $j$ must be at least the service start time at $i$, plus the service time $s_i$ to $i$ and the travel time between $i$ and $j$. This constraint keeps track of the time evolution along the vehicle's route.
    $$\sum_{i \in L}[z_{ij} + (s_i + t_{ij})x_{ij}] \leq \sum_{i \in L} z_{ji} \quad \forall j \in C$$

In [248]:
model_flow.addConstrs(
    (
        gp.quicksum(
            z[i, j] + (serviceTimes[i] + travelTimes[i, j]) * x[i, j]
            for i in locations
            if (i, j) in connections
        )
        <= z.sum(j, "*")
        for j in customers
    ),
    name="flowConservation",
)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

- The subsequent constraints impose that the start and end time of the service must be limited by the respective time slot $(a, b)$.
    $$z_{ij}\geq a_ix_{ij} \quad \forall i,j \in L, i \neq j$$
    $$z_{ij}\leq b_ix_{ij} \quad \forall i,j \in L, i \neq j$$

In [249]:
model_flow.addConstrs(
    (
        z[i, j] >= timeWindows[i][0] * x[i, j]
        for i in customers
        for j in locations
        if i != j
    ),
    name="timeWindowStart",
)
model_flow.addConstrs(
    (
        z[i, j] <= timeWindows[i][1] * x[i, j]
        for i in customers
        for j in locations
        if i != j
    ),
    name="timeWindowEnd",
)

{(1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


So, we can now optimize the model and plot the solution:

In [250]:
model_flow.Params.OutputFlag = 0

model_flow.optimize()

if model_flow.SolCount >= 1:

    print("Objective Value: ", model_flow.objVal)

    usedConnections = [(i, j) for (i, j) in x.keys() if x[i, j].X > 0.5]

    # create a dict for the next customer based on the current one
    # (note that the depot in general has multiple outgoing connections)
    nextCustomer = {}
    for (i, j) in usedConnections:
        if i == 0:
            if 0 not in nextCustomer.keys():
                nextCustomer[0] = []
            nextCustomer[0].append(j)
        else:
            nextCustomer[i] = j

    print(f"Solution contains {len(nextCustomer[0])} routes:")
    routeNumber = 0
    visitedCustomers = [False] * (numCustomers + 1)
    for firstCustomer in nextCustomer[0]:
        print(f"Route {routeNumber}: 0 -> ", end="")
        vehicleLoad = 0
        #time = travelTimes[0, firstCustomer]
        #violatedTimeWindows = False
        currentCustomer = firstCustomer
        while currentCustomer != 0:
            print(f"{currentCustomer} (L:{vehicleLoad}) -> ", end="")
            visitedCustomers[currentCustomer] = True
            vehicleLoad += demands[currentCustomer]
            #time = max(time, timeWindows[currentCustomer][0])
            #if time > timeWindows[currentCustomer][1]:
                #violatedTimeWindows = True
            #time += (
                #serviceTimes[currentCustomer]
                #+ travelTimes[currentCustomer, nextCustomer[currentCustomer]]
            #)
            currentCustomer = nextCustomer[currentCustomer]
        print(f"0 (L:{vehicleLoad}/{vehicleCapacity})")
        if vehicleLoad > vehicleCapacity:
            print("Vehicle capacity is exceeded!")
        #if violatedTimeWindows:
            #print("Time windows are violated!")
        routeNumber += 1

    print("Unvisited customers: ", end="")
    for c in customers:
        if visitedCustomers[c] == False:
            print(f"{c}, ", end="")
    print()
    for i in locations:
        for j in locations:
            if i != 0:
                if i != j:
                    if model_flow.getAttr('x',x)[i, j] == 1:
                        print(f"Client {i}: service start time = {model_flow.getAttr('x', z)[i, j]}, time window = {timeWindows[i]}")

Objective Value:  51.0
Solution contains 2 routes:
Route 0: 0 -> 1 (L:0) -> 3 (L:4) -> 5 (L:8) -> 0 (L:11/30)
Route 1: 0 -> 2 (L:0) -> 4 (L:5) -> 0 (L:8/30)
Unvisited customers: 
Client 1: service start time = 12.0, time window = (12, 26)
Client 2: service start time = 9.0, time window = (9, 24)
Client 3: service start time = 42.0, time window = (42, 56)
Client 4: service start time = 71.0, time window = (60, 71)
Client 5: service start time = 83.0, time window = (71, 83)


# Module 3
The Module 3 is related to the *Last Mile Delivery Problem with Parcel Lockers*. This problem is a variant of VRP where vehicles have to deliver parcels to a set of consumers that can be served directly at their home or, as an alternative, to a locker station.

So, the objective of the problem is to decide which locker stations to open to minimize the delivery time.

For this reason, we have to update the previous notation. Now we have two sets of nodes:
- $N_C$ is the set of customers
- $N_L$ is the set of lockers

So the complete direct graph will be $G = (N, A)$ where $N = {0} \cup N_C \cup N_L$ and $A$ is the arc set.

In addition, the problem requires that a customer is willing to travel from home to collecthis package iff the distance between the customer and the locker is lower or equal to $d_{max}$.

To separately account for arcs travelled by consumers, we define as $N_L^c \in N_L$ the subset of potential parcel lockers located at a distance lower than or equal to $d_{max}$ from customer $c$.

We can assume that locker stations have unbounded capacity.

In [251]:
N_C = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # clients
N_L = [11, 12, 13]  # lockers
N = [0] + N_C + N_L  # N = {0} ∪ N_C ∪ N_L (0 is the depot)
num_nodes = len(N)
A = [(i, j) for i in N for j in N if i != j] # set of arcs
vehicles = [1, 2, 3]
K = len(vehicles)
t = {0: 0, 1: 2, 2: 3, 3: 2, 4: 1, 5: 2, 6: 4, 7: 2, 8: 3, 9: 2, 10: 1, 11: 3, 12: 5, 13: 2} # service time

# maximum distance
d_max = 10

# Distances between nodes
np.random.seed(0)  # Per la riproducibilità
distance_matrix = {}

for i in range(num_nodes):
    for j in range(i + 1, num_nodes):
        distance = np.random.randint(1, 31)  # Distanza casuale tra 1 e 30
        distance_matrix[(N[i], N[j])] = distance
        distance_matrix[(N[j], N[i])] = distance  # Distanza bidirezionale

# Function to return the distance between two nodes
def distance(i, j):
    if i == j:
        return 0
    return distance_matrix.get((i, j), distance_matrix.get((j, i), float('inf')))

time_windows = {
    1: (3, 40),
    2: (2, 40),
    3: (6, 40),
    4: (5, 40),
    5: (2, 40),
    6: (0, 40),
    7: (4, 40),
    8: (0, 40),
    9: (0, 40),
    10: (0, 40)
}

As in the previous module we can define the following **decision variables**:
- $x_{ij} = 
\begin{cases} 
1 & \text{if a vehicle traverses arc (i, j)} \\
0 & \text{otherwise}
\end{cases}
$
- a continuous variable $z_{ij}$ indicating the time of arrival of a vehicle at node $j$ when arriving from $i$
- for each set of nodes $S \in N$, where $N$ is the total set of nodes, let $$\delta^+(S) = \{(i, j) \in A : i \in S, j \notin S\}$$ $$\delta^-(S) = \{(i, j) \in A : i \notin S, j \in S\}$$
be the set of arcs leaving and entering set S. So, if $S = {i}$ then $\delta^+(S) = \delta^+({i})$
- The decision to open a locker at any location $l \in N_L$ is regulated by the binary variable 
$$y_{l} = 
\begin{cases} 
1 & \text{if the locker is opened} \\
0 & \text{otherwise}
\end{cases}
$$
- $w_{cl} = 
\begin{cases} 
1 & \text{if customer c travels to locker l} \\
0 & \text{otherwise (the consumer receives the parcel directly at
home)}
\end{cases}
$

In [252]:
model = gp.Model("VRPPL")

# Decision Variables

# 1 if arc (i, j) is travelled
x = model.addVars(N, N, vtype=GRB.BINARY, name="x")

# Arriving time at node j
z = model.addVars(N, N, vtype=GRB.CONTINUOUS, name="z")

# 1 if locker i is opened
y = model.addVars(N_L, vtype=GRB.BINARY, name="y")

# 1 if client c goes to locker i
w = model.addVars(N_C, N_L, vtype=GRB.BINARY, name="w")

# Set of accessible lockers for each customer, respecting d_max
N_L_c = {c: [l for l in N_L if distance(c, l) <= d_max] for c in N_C}

# Objective function
model.setObjective(
    gp.quicksum(distance(i, j) * x[i, j] for i in N for j in N if i != j),
    GRB.MINIMIZE
)

So, we can plot, for each customer, the set of lockers that can be reached

In [253]:
N_L_c

{1: [],
 2: [13],
 3: [],
 4: [12],
 5: [13],
 6: [11, 12, 13],
 7: [11, 12],
 8: [11, 12, 13],
 9: [12, 13],
 10: []}

We can now formulate the **constraints**:
- This constraints impose that each customer $c \in N_C$ is served either at home ($\sum_{l \in N_L^c}w_{cl} = 0$) or at a locker station ($\sum_{l \in N_L^c}w_{cl} = 1$), hence customers are included in a tour only when they are not travelling to any locker.
$$\sum_{(i,c) \in \delta^-(c)}x_{ic} = \sum_{(c,i) \in \delta^+(c)}x_{ci} = 1 - \sum_{l \in N_L^c}w_{cl} \quad \forall c \in N_C$$

In [254]:
for c in N_C:
    # Primo membro: somma degli archi entranti nel cliente c
    incoming_arcs = gp.quicksum(x[i, c] for i in N if (i, c) in A)  # δ^-(c)
    
    # Secondo membro: somma degli archi uscenti dal cliente c
    outgoing_arcs = gp.quicksum(x[c, i] for i in N if (c, i) in A)  # δ^+(c)
    
    # Somma delle variabili w[c, l] per i locker che possono servire il cliente c
    lockers_used = gp.quicksum(w[c, l] for l in N_L_c[c])
    
    # Vincolo: cliente servito direttamente o tramite locker
    model.addConstr(incoming_arcs == outgoing_arcs, name=f"flow_conservation_c_{c}")
    model.addConstr(incoming_arcs == 1 - lockers_used, name=f"served_directly_or_locker_c_{c}")

- This constraint set the inclusion of lockers into routes, stating that a vehicle needs to visit a locker only if it is open.
$$\sum_{(i,l)\in \delta^-(l)}x_{il} = \sum_{(l,i)\in \delta^+(l)}x_{li} = y_l \quad \forall l \in N_L$$

In [255]:
for l in N_L:
    # Somma degli archi che entrano nel locker l
    model.addConstr(
        gp.quicksum(x[i, l] for i in N if i != l) == y[l],
        name=f"visit_locker_in_{l}"
    )

    # Somma degli archi che escono dal locker l
    model.addConstr(
        gp.quicksum(x[l, i] for i in N if i != l) == y[l],
        name=f"visit_locker_out_{l}"
    )

- This constraint ensures that at most $K$ vehicles are used
$$\sum_{(0,j)\in \delta^+(0)}x_{0j} = \sum_{(j,0)\in \delta^-(0)}x_{j0} \le |K|$$

In [256]:
model.addConstr(
    gp.quicksum(x[0, j] for j in N if j != 0) == gp.quicksum(x[j, 0] for j in N if j != 0),
    name="balance_vehicles_in_out"
)

model.addConstr(
    gp.quicksum(x[0, j] for j in N if j != 0) <= K,
    name="max_vehicles"
)

<gurobi.Constr *Awaiting Model Update*>

- This constraint statrs that a client can travel to a locker only if it has been opened and served by one vehicle route
$$w_{cl} \le y_l \quad \forall c \in N_C, l \in N_L^c$$

In [257]:
for c in N_C:
    for l in N_L_c[c]:  # Solo i lockers accessibili dal cliente c
        model.addConstr(
            w[c, l] <= y[l],
            name=f"client_to_locker_{c}_{l}"
        )

- This constraint determines the arrival time at two consecutive nodes, thus working as sub - tour elimination constraints. In particular, if node $j$ is visited immediately after node $i$, the time elapsed between the arrival in the two nodes is equal to the time $t_{ij}$ needed to travel between the two nodes plus the service time at node $i$.
$$\sum_{(i,j)\in\delta^+(i)}z_{ij} - \sum_{(j,i)\in\delta^-(i)}z_{ji} = \sum_{(i,j)\in\delta^+(i)}(t_{ij} + t_i)x_{ij} \quad \forall i \in N_L \cup N_C$$

In [258]:
for i in N_L + N_C:
    model.addConstr(
        gp.quicksum(z[i, j] for j in N if (i, j) in A) - 
        gp.quicksum(z[j, i] for j in N if (j, i) in A) ==
        gp.quicksum((distance(i, j) + t[i]) * x[i, j] for j in N if (i, j) in A),
        name=f"arrival_time_constraint_{i}"
    )

- This constraint sets the lower and upper bound of variable $z_{ij}$. If arc $(i,j)$ is traversed, then the arrival time at node $j$ must be greater than the time required to leave the depot and serve the customer $i$ ($t_{0,i} + t_{i}$) and lower than the allowed tour length ($T$) minus the time required to serve the client $j$ and return to the depot ($t_j + t_{j0}$)
$$(t_{0i} + t_{ij} + t_i)x_{ij} \le z_{ij} \le (T - t_{j0} - t_j)x_{ij} \quad \forall (i,j) \in A$$

In [259]:
T = 100  # lunghezza massima del tour

# Aggiungi vincoli per ogni arco (i, j) ∈ A
for i, j in A:
    if i != j:
        # Vincolo inferiore: (t_{0i} + t_{ij} + t_i)x_{ij} ≤ z_{ij}
        model.addConstr(
            (distance(0, i) + distance(i, j) + t[i]) * x[i, j] <= z[i, j],
            name=f"lower_bound_z_{i}_{j}"
        )
        
        # Vincolo superiore: z_{ij} ≤ (T - t_{j0} - t_j)x_{ij}
        model.addConstr(
            z[i, j] <= (T - distance(j, 0) - t[j]) * x[i, j],
            name=f"upper_bound_z_{i}_{j}"
        )

- This constraint ensures that the time needed to travel from the depot to any visited node $i$ (when $x_{0i} = 1$) is equal to $t_{0i}$
$$z_{0i} = t_{0i}x_{0i} \quad \forall i \in N_L \cup N_C$$

In [260]:
for i in N_L + N_C:
    model.addConstr(z[0, i] == distance(0, i) * x[0, i], name=f"time_depot_to_{i}")

- This constraint ensures that the arriving time to each customer is in the range $[a, b]$

In [261]:
for c in N_C:
    a_c, b_c = time_windows[c]
    
    # Vincolo: il veicolo non può arrivare prima di a_c
    model.addConstr(
        gp.quicksum(z[i, c] for i in N if (i, c) in A) >= a_c, 
        name=f"time_window_lower_bound_{c}"
    )
    
    # Vincolo: il veicolo non può arrivare dopo b_c
    model.addConstr(
        gp.quicksum(z[i, c] for i in N if (i, c) in A) <= b_c,
        name=f"time_window_upper_bound_{c}"
    )

We define a function in order to print the final route.

In [262]:
def print_route_and_objective(solution_x, solution_z, N, N_C, N_L, model, time_windows):
    # Teniamo traccia di quali nodi sono già stati visitati
    visited = set()
    
    # Inizia dal deposito (nodo 0)
    current_node = 0
    route = [current_node]
    
    while True:
        # Trova il prossimo nodo che è collegato al nodo corrente con x_{current_node, j} = 1
        next_node = None
        for j in N:
            if current_node != j and solution_x[current_node, j] > 0.5 and j not in visited:
                next_node = j
                break
        
        if next_node is None:
            # Nessun nodo successivo trovato, ritorna al deposito
            break
        
        # Aggiungi il nodo successivo alla rotta e segna come visitato
        route.append(next_node)
        visited.add(next_node)
        current_node = next_node
    
    # Aggiungi il ritorno al deposito se necessario
    if current_node != 0:
        route.append(0)
    
    # Stampa il percorso del veicolo
    print("Optimal Route:")
    for i, node in enumerate(route):
        if node == 0 and i != 0:  # Se arrivi al nodo 0 e non è il primo nodo
            print("0")  # Vai a capo dopo il nodo 0 e stampa "0" per il nuovo percorso
            if i != len(route) - 1:  # Se non è l'ultimo nodo, inizia il nuovo percorso con "0 ->"
                print("0 -> ", end="")
        else:
            if i == len(route) - 1:  # Se è l'ultimo nodo non mettere la freccia
                print(node, end="")
            else:
                print(node, end=" -> ")


    
    # Stampa il valore della funzione obiettivo
    print(f"Objective vfunction value: {model.objVal}")
    
    # Stampa i tempi di arrivo confrontati con le finestre temporali
    print("\nCustomer arrival times and comparison with time windows:")
    for c in N_C:
        a_c, b_c = time_windows[c]
        arrival_time = None
        for i in N:
            if (i, c) in solution_z and solution_x[i, c] == 1:
                arrival_time = solution_z[i, c]
                break
        if arrival_time is not None:
            print(f"Client {c}: arrival time = {arrival_time}, time windows = [{a_c}, {b_c}]")
            if arrival_time < a_c:
                print(f" -> Arrivo anticipato rispetto alla finestra temporale!")
            elif arrival_time > b_c:
                print(f" -> Arrivo in ritardo rispetto alla finestra temporale!")
        else:
            print(f"Client {c}: no arrival recorded")

    # Stampa quali clienti vanno a un locker
    print("\nCustomers going to a locker:")
    for c in N_C:
        for l in N_L_c[c]:
            if w[c, l].X > 0.5:  # Se il cliente c va al locker l
                print(f"Client {c} goes to locker {l}")

We can optimize the model and print the solutions:

In [263]:
model.Params.OutputFlag = 0

model.optimize()

# Dopo l'ottimizzazione
if model.status == GRB.OPTIMAL:
    # Estrai i valori ottimali delle variabili x_{ij} e z_{ij}
    solution_x = model.getAttr('x', x)
    solution_z = model.getAttr('x', z)
    
    # Stampa il tragitto del veicolo, i tempi di arrivo e il confronto con le time windows
    print_route_and_objective(solution_x, solution_z, N, N_C, N_L, model, time_windows)


Optimal Route:
0 -> 5 -> 13 -> 2 -> 7 -> 6 -> 3 -> 0
0 -> 8 -> 1 -> 10 -> 4 -> 0
Objective vfunction value: 75.0

Customer arrival times and comparison with time windows:
Client 1: arrival time = 13.0, time windows = [3, 40]
Client 2: arrival time = 17.0, time windows = [2, 40]
Client 3: arrival time = 37.0, time windows = [6, 40]
Client 4: arrival time = 34.0, time windows = [5, 40]
Client 5: arrival time = 4.0, time windows = [2, 40]
Client 6: arrival time = 32.0, time windows = [0, 40]
Client 7: arrival time = 29.0, time windows = [4, 40]
Client 8: arrival time = 8.0, time windows = [0, 40]
Client 9: no arrival recorded
Client 10: arrival time = 23.0, time windows = [0, 40]

Customers going to a locker:
Client 9 goes to locker 13
